# <span style="color:#0b486b">  FIT5215: Deep Learning (2023)</span>
***
*CE/Lecturer (Clayton):*  **Dr Trung Le** | trunglm@monash.edu <br/>
*Lecturer (Malaysia):*  **Dr Lim Chern Hong** | lim.chernhong@monash.edu <br/>  <br/>
*Tutor:*  **Mr Tuan Nguyen**  \[tuan.ng@monash.edu \] | **Dr Binh Nguyen** \[binh.nguyen1@monash.edu \] | **Dr Qiuhong Ke** \[Qiuhong.Ke@monash.edu  \] 
<br/> <br/>
Faculty of Information Technology, Monash University, Australia
***

# <span style="color:#0b486b">Tutorial 5b: Convolutional Neural Network with MiniVGG</span><span style="color:red;  font-size: 18px">***** (highly important)</span> #
**This tutorial applies CNN to a more real-world setting using TensorFlow using the `CIFAR-10` dataset. It will cover the following**:
1. ***How to implement a simplified version of a real-world CNN named `MiniVGG` to classify the `CIFAR-10` dataset using TF 2.x.***

VGGNet, (aka VGG), was first introduced by Simonyan and Zisserman in their 2014 paper, *Very Deep Learning Convolutional Neural Networks for Large-Scale Image Recognition*. 
The work was the first to demonstrate that architecture with very small (3×3) filters can be successfully trained for very deep networks (16-19 layers) and obtain state-of-the-art classification on the challenging ImageNet classification challenge. <br/>

The VGG family of Convolutional Neural Networks can be characterized by two key components:
1. *All CONV layers in the network using only **3×3** filters.*
2. *Stacking multiple CONV => RELU layer sets (where the number of consecutive CONV => RELU layers normally increases as we go deeper) before applying a POOL operation.* <br/>

Due to the size of this architecture which compromises the running time, this tutorial will use `MiniVGG` which is a simplified version of `VGG` including its architecture, implementation, and application to the `CIFAR-10` dataset.

---

## <span style="color:#0b486b">I. MiniVGG for the CIFAR-10 dataset</span> ##

### <span style="color:#0b486b">I.1. Architecture of MiniVGG </span>
The following figure and table show the concrete architecture of `MiniVGG`. As mentioned before, `MiniVGG` only uses $3\times3$ filters. In addition, in the `MiniVGGNet`, we apply the batch norm technique to regularize the training and allow the network to be trained with a higher learning rate.

<img src='images/MiniVGG_Table.png' align='center' width=300/>

### <span style="color:#0b486b">I.2. Implementation of MiniVGG </span>

#### <span style="color:#0b486b">I.2.1 Import necessary packages and modules </span>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, models, layers
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
np.random.seed(123)
tf.random.set_seed(123)

#### <span style="color:#0b486b">I.2.2 Download, prepare and preprocess CIFAR-10 dataset </span>

The CIFAR-10 dataset contains 60,000 color images in 10 classes, with 6,000 images in each class. The dataset is divided into 50,000 training images and 10,000 testing images. The classes are mutually exclusive and there is no overlap between them.

In [ ]:
(X_train_full, y_train_full) , (X_test_full, y_test_full) = datasets.cifar10.load_data()
X_train_full, X_test_full = X_train_full/255.0, X_test_full/255.0
print(X_train_full.shape, y_train_full.shape)
print(X_test_full.shape, y_test_full.shape)

Shuffle the entire dataset.

In [ ]:
train_val_idx = np.random.permutation(X_train_full.shape[0])
test_idx = np.random.permutation(X_test_full.shape[0])

Create a tiny `CIFAR-10` for a demonstration.

In [ ]:
n_train, n_valid, n_test = 5000, 5000, 5000 
X_train, y_train = X_train_full[train_val_idx][:n_train], y_train_full[train_val_idx][:n_train]
X_valid, y_valid = X_train_full[train_val_idx][n_train:n_train+n_valid], y_train_full[train_val_idx][n_train:n_train+n_valid]
X_test, y_test = X_test_full[test_idx][:n_test], y_test_full[test_idx][:n_test]
y_train = y_train.reshape(-1)
y_valid = y_valid.reshape(-1)
y_test = y_test.reshape(-1)
print('Training set', X_train.shape, y_train.shape)
print('Validation set', X_valid.shape, y_valid.shape)
print('Test set', X_test.shape, y_test.shape)

#### <span style="color:#0b486b">I.2.3 Visualize CIFAR-10 images </span>

In [ ]:
import math
def visualize_data(images, categories, images_per_row = 8):
    class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    n_images = len(images)
    n_rows = math.ceil(float(n_images)/images_per_row)
    fig = plt.figure(figsize=(1.5*images_per_row, 1.5*n_rows))
    fig.patch.set_facecolor('white')
    for i in range(n_images):
        plt.subplot(n_rows, images_per_row, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(images[i])
        class_index = categories[i]
        plt.xlabel(class_names[class_index])
    plt.show()

In [ ]:
visualize_data(X_train[0:32], y_train[0:32])

#### <span style="color:#0b486b">I.2.4 Build up MiniVGG network </span>

In [ ]:
def create_vgg():
    vgg_model = models.Sequential()
    vgg_model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(32,32,3)))
    vgg_model.add(layers.BatchNormalization(momentum=0.9))
    vgg_model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu'))
    vgg_model.add(layers.BatchNormalization(momentum=0.9))
    vgg_model.add(layers.MaxPool2D(pool_size=(2,2)))  #downscale the image size by 2
    vgg_model.add(layers.Dropout(rate=0.25))  #deactivate 25% of neurons for each feed-forward
    vgg_model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    vgg_model.add(layers.BatchNormalization(momentum=0.9))
    vgg_model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    vgg_model.add(layers.BatchNormalization(momentum=0.9))
    vgg_model.add(layers.MaxPool2D(pool_size=(2,2)))  #downscale the image size by 2
    vgg_model.add(layers.Dropout(rate=0.25))  #deactivate 25% of neurons for each feed-forward
    vgg_model.add(layers.Flatten())
    vgg_model.add(layers.Dense(512, activation='relu'))
    vgg_model.add(layers.BatchNormalization(momentum=0.9))
    vgg_model.add(layers.Dropout(rate=0.5)) 
    vgg_model.add(layers.Dense(10, activation='softmax')) #ten classes in CIFAR-10
    return vgg_model

In [ ]:
vgg = create_vgg()
vgg.summary()

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.001)
vgg.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
no_aug_history = vgg.fit(x=X_train, y=y_train, batch_size=32, epochs=30, validation_data=(X_valid, y_valid))

We now can evaluate the performance on the CIFAR-10 test set.

In [ ]:
test_loss, test_acc = vgg.evaluate(X_test,  y_test, verbose=1, batch_size=64)
print("Test acc is {}".format(test_acc))

### <span style="color:#0b486b">I.2.5 Plot training loss and accuracy</span> ###

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(no_aug_history.history['accuracy'], label='train accuracy', color='green', marker="o")
ax1.plot(no_aug_history.history['val_accuracy'], label='valid accuracy', color='blue', marker = "v")
ax2.plot(no_aug_history.history['loss'], label = 'train loss', color='orange', marker="o")
ax2.plot(no_aug_history.history['val_loss'], label = 'valid loss', color='red', marker = "v")
ax1.legend(loc=3)

ax1.set_xlabel('Epochs')
ax1.set_ylabel('Accuracy', color='g')
ax2.set_ylabel('Loss', color='b')
ax2.legend(loc=4)
plt.ylim([0.0, 5])
plt.show()

## <span style="color:#0b486b">II. MiniVGG with data augmentation</span> ##

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(horizontal_flip=True, width_shift_range=0.1, height_shift_range=0.1)
datagen.fit(X_train)

it = datagen.flow(X_train, y_train, batch_size=32)
batch_images, batch_labels = next(it)

visualize_data(batch_images, batch_labels)

In [ ]:
vgg = create_vgg()
opt = keras.optimizers.Adam(learning_rate=0.001)
vgg.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
steps = len(X_train) // 32
with_aug_history = vgg.fit(it, epochs=30, steps_per_epoch=steps, validation_data=(X_valid, y_valid))

In [ ]:
test_loss, test_acc = vgg.evaluate(X_test,  y_test, verbose=1, batch_size=64)
print("Test acc is {}".format(test_acc))

We can observe an improvement when evaluating on the testing set.

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(with_aug_history.history['accuracy'], label='train accuracy', color='green', marker="o")
ax1.plot(with_aug_history.history['val_accuracy'], label='valid accuracy', color='blue', marker = "v")
ax2.plot(with_aug_history.history['loss'], label = 'train loss', color='orange', marker="o")
ax2.plot(with_aug_history.history['val_loss'], label = 'valid loss', color='red', marker = "v")
ax1.legend(loc=3)

ax1.set_xlabel('Epochs')
ax1.set_ylabel('Accuracy', color='g')
ax2.set_ylabel('Loss', color='b')
ax2.legend(loc=4)
plt.ylim([0.0, 5])
plt.show()

## <span style="color:#0b486b">III. Visualize filters and feature maps of MiniVGG</span> ##

Visualizing the filters and feature maps is important to help us understand what is learned by trained CNNs. We expect that low layers can capture low-level features such as edges, pixels, and corners, middle layers can capture mid-level features such as circles, boxes, triangles, while high layers can capture high-level features such as objects. According to the operational principle of CNNs, features captured by higher layers are a combination of those in lower layers. For example, edges combine into boxes, and boxes and triangles, in turn, combine into more complicated objects.

We now explore how to visualize the filters and feature maps for our trained MiniVGG network.

We first play around with how to get all layers in a TF keras model and how to extract information from a specific layer.

In [ ]:
for i, layer in enumerate(vgg.layers):
    print("Layer {}: {}".format(i, layer.name))

In what follows, we examine all Conv2D layers in our network and print out their filters. Recall that the set of filters is represented as a $4D$ tensor with the shape $[fil\_height, fil\_width, fil\_depth, num\_filters]$.

In [ ]:
for layer in vgg.layers:
    if isinstance(layer, tf.keras.layers.Conv2D):
        print("Layer name: {}, shape: [{}, {}]".format(layer.name, layer.filters, layer.kernel_size))
        print(layer.get_weights()[0].shape)

We now visualize a filter in a Conv2D layer of a specific model. Note that each filter is a 3D tensor with shape $[fil\_height, fil\_width, fil\_depth]$.

In [ ]:
import math
def visualize_filter(model, layer_name= None, filter_index=1, n_cols = 8):
    layer = model.get_layer(layer_name)
    assert isinstance(layer, tf.keras.layers.Conv2D), "The current layer is not a Conv2D layer"
    filters = layer.get_weights()[0]  #4D tensor of all filters
    assert filter_index <= filters.shape[-1], "The filter index is out of range"
    one_filter = filters[:,:,:, filter_index-1]  #the filter we need to visualize
    fil_depth = one_filter.shape[-1]
    n_rows = math.ceil(fil_depth/n_cols)
    fig = plt.figure(figsize=(n_cols*1.5, n_rows*1.5))
    plt.axis("off")
    for i in range(fil_depth):
        plt.subplot(n_rows, n_cols, i+1)
        plt.imshow(one_filter[:,:,i], cmap="gray")
        plt.xlabel("{} of {}".format(i, filter_index))
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)

In [ ]:
print("We visualize the 5th filter in the filters of the convolutional layer {} (Layer 2)".format(vgg.layers[2].name))
visualize_filter(vgg, vgg.layers[2].name, filter_index=5)

In [ ]:
print("We visualize the 5th filter in the filters of the convolutional layer {} (Layer 6)".format(vgg.layers[6].name))
visualize_filter(vgg, vgg.layers[6].name, filter_index=5)

 **<span style="color:red">Exercise 1</span>:** Expand the function `visualize_filter` so that if `filter_index= None`, we visualize all filters of the layer.

In [ ]:
def visualize_filter_all(model, layer_name=None, n_cols=8):
    layer = model.get_layer(layer_name)
    assert isinstance(layer, tf.keras.layers.Conv2D), "The current layer is not a Conv2D layer"
    filters = layer.get_weights()[0]  # 4D tensor of all filters
    for filter_index in range(filters.shape[-1]):
        visualize_filter(model, layer_name, filter_index, n_cols=8)

In [ ]:
visualize_filter_all(vgg, vgg.layers[2].name)  # Assume we visualize layer 2

Now let us visualize the feature maps before and after training via feeding an image to the network.

Choose an arbitrary image from the training set.

In [ ]:
plt.figure(figsize=(2, 2))
rand_img = X_train[5]
plt.imshow(rand_img)

In [ ]:
def visualize_feature_maps(input_img= None, model= None, layer_name= None, n_cols= 8):
    intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
    if len(input_img.shape)==3:
        input_img = tf.expand_dims(input_img, axis =0)
    intermediate_output = intermediate_model.predict(input_img)
    n_feature_maps = intermediate_output.shape[-1]
    n_rows = math.ceil(n_feature_maps/n_cols)
    fig = plt.figure(figsize=(n_cols*1.5, n_rows*1.5))
    plt.axis("off")
    for i in range(n_feature_maps):
        plt.subplot(n_rows, n_cols, i+1)
        plt.imshow(intermediate_output[0,:,:, i], cmap="gray")
        plt.xlabel("{} of {}".format(i, n_feature_maps), fontsize=10)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)

**Visualizing feature maps before training.**

In [ ]:
vgg_init = create_vgg()
print("Before the training:")
print("Feature maps at layer {}.".format(vgg_init.layers[2].name))
visualize_feature_maps(rand_img, vgg_init, vgg_init.layers[2].name)

**Visualizing feature maps after training.**

In [ ]:
print("Feature maps at 0th layer.")
visualize_feature_maps(rand_img, vgg, vgg.layers[0].name)

In [ ]:
print("Feature maps at 4th layer.")
visualize_feature_maps(rand_img, vgg, vgg.layers[4].name)

In [ ]:
print("Feature maps at 8th layer.")
visualize_feature_maps(rand_img, vgg, vgg.layers[8].name)

**<span style="color:red">Exercise 2</span>:** Choose another arbitrary image and expand the function `visualize_feature_maps` to visualize the original image (as the first image) along with its feature maps as above. The expected results should display (`n_feature_maps` + 1) figures. The caption for the image is its class name (true label).

In [ ]:
def visualize_feature_maps(input_img=None, true_label=None, model=None, layer_name=None, n_cols=8):
    class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    if len(input_img.shape)==3:
        batch_input_img = tf.expand_dims(input_img, axis =0)  #make it a 4D tensor with batch_size=1
    intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
    intermediate_output = intermediate_model.predict(batch_input_img)
    n_feature_maps = intermediate_output.shape[-1]
    n_images = math.floor(n_feature_maps) + 1
    n_rows = math.ceil(n_images/n_cols)
    fig = plt.figure(figsize=(n_cols*1.5, n_rows*1.5))
    plt.axis("off")
    for i in range(n_images):
        plt.subplot(n_rows, n_cols, i+1)
        img = input_img if i==0 else intermediate_output[0,:,:, i-1]
        label = "{}".format(class_names[true_label]) if i==0 else "{} of {}".format(i, n_feature_maps)
        if i==0:
            plt.imshow(img)
        else:
            plt.imshow(img, cmap= "gray")
        plt.xlabel(label, fontsize= 10)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)

In [ ]:
visualize_feature_maps(X_train[0], y_train[0], vgg, vgg.layers[6].name)

**<span style="color:red">Exercise 3</span>:** In the lecture and tutorial of week 7, we will study how to implement SOTA CNN networks for example RESNET, DenseNet and also how to perform transfer learning and fine-tuning a SOTA pretrained CNN to fit a specific and tiny dataset. It is appealing now to investigate what is learned by a SOTA pretrained CNN by visualizing its filters. You will have more sense of why CNNs work well for visual data. Your task is to apply the above code to visualize the filters of a pretrained SOTA CNN. Please use the following code as a good start.

In [ ]:
from tensorflow.keras.applications import VGG16

In [ ]:
print("Loading network...")
base_model = VGG16(weights="imagenet", include_top=False, input_tensor = layers.Input(shape=(32,32,3)))
print("Showing layers...")
# loop over the layers in the network and display them to the console
for (i, layer) in enumerate(base_model.layers):
    print("Layer {}: {}".format(i, layer.__class__.__name__))

In [ ]:
print(base_model.layers[2].weights[0].shape)

We can visualize a filter using `visualize_filter` or visualize all filters of a convolutional layer using `visualize_filter_all` in the solution of Exercise 1.

In [ ]:
visualize_filter(base_model, 'block1_conv2', filter_index=0)  

---
### <span style="color:#0b486b"> <div  style="text-align:center">**THE END**</div> </span>